# Regression Model Exploratory

#### Load

In [2]:
from IPython import get_ipython
if get_ipython() is not None:
    get_ipython().run_line_magic("load_ext", "autoreload")
    get_ipython().run_line_magic("autoreload", "2")

import model_funcs as mf

import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

from centraljersey import merge, preprocess

from centraljersey.models import PredictionModels
model = PredictionModels()
df = model.df_tracts

culture_variables = [
    "dunkin_id",
    "wawa_id",
    "giants_or_jets",
    "pork_roll",
    "calm-no-l",
    "almond-no-l",
    "forward-no-r",
    "drawer",
    "gone-don",
]

census_variables = [
    "white_pop",
    "black_pop",
    "asian_pop",
    "occu_Agricul/fish/mining/forest",
    "occu_Construction",
    "occu_Manufacturing",
    "occu_Wholesale trade",
    "occu_Retail trade",
    "occu_transport/warehouse/utils",
    "occu_Information",
    "occu_finance/insurance/realestate",
    "occu_administrative",
    "occu_educational/healthcare/social",
    "occu_arts/entertainment/foodservices",
    "occu_public administration",
    "occu_management, business",
    "occu_Service occupations:",
    "occu_Sales and office occupations:",
    "occu_Natural resources, construction",
    "occu_production/transport/materials",
    "income_150k+",
    "pob_foreign_born",
    "edu_college",
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/chansoo/projects/centraljersey/centraljersey/models.py:17: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  self.df_county = self.merger.df_counties


#### Train/Test

In [58]:
model_cols = [
    'dunkin_id',
    'wawa_id',
    'giants_or_jets',
    'pork_roll',
    'calm-no-l',
    'almond-no-l',
    'forward-no-r',
    'drawer',
    'gone-don',
    # 'white_pop',
    'black_pop',
    'asian_pop',
    # 'occu_Agricul/fish/mining/forest',
    # 'occu_Construction',
    # 'occu_Manufacturing',
    # 'occu_Wholesale trade',
    # 'occu_Retail trade',
    # 'occu_transport/warehouse/utils',
    # 'occu_Information',
    # 'occu_finance/insurance/realestate',
    # 'occu_administrative',
    # 'occu_educational/healthcare/social',
    # 'occu_arts/entertainment/foodservices',
    # 'occu_public administration',
    # 'occu_management, business',
    # 'occu_Service occupations:',
    # 'occu_Sales and office occupations:',
    # 'occu_Natural resources, construction',
    # 'occu_production/transport/materials',
    'income_150k+',
    'pob_foreign_born',
    'edu_college'
]

In [3]:
mf.split_data(df=df, vars=culture_variables)

AttributeError: module 'statsmodels' has no attribute 'add_constant'

In [51]:
logit_model = sm.Logit(y_train, X_train)
result = logit_model.fit()

         Current function value: inf
         Iterations: 35


/home/chansoo/projects/centraljersey/.venv/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/chansoo/projects/centraljersey/.venv/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/chansoo/projects/centraljersey/.venv/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/chansoo/projects/centraljersey/.venv/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, 

LinAlgError: Singular matrix